In [215]:
from datasets import load_dataset
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
from rouge_score import rouge_scorer

In [2]:
dataset = load_dataset('mlsum', 'fr')

Reusing dataset mlsum (C:\Users\maxim\.cache\huggingface\datasets\mlsum\fr\1.0.0\77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


  0%|          | 0/3 [00:00<?, ?it/s]

In [228]:
article = dataset['train']['text'][4]
summary = dataset['train']['summary'][4]
title = dataset['train']['title'][4]
print(article, summary, title, sep='\n\n')

Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes, dans le Gard, dans un incendie qui s'est déclenché vendredi 1er janvier au petit matin. Le feu, dont on ignore l'origine pour l'instant, a pris au sixième et dernier étage d'un immeuble. "A l'arrivée des pompiers, trois personnes étaient décédées dans un appartement, et deux autres dans un appartement voisin par intoxication", a expliqué, sur i-Télé, le directeur de cabinet du préfet du Gard. On dénombre également "treize blessés, dont trois graves. Une personne dans le coma a été transférée à Marseille", a-t-il ajouté. Les secours ont été prévenus vers 5 heures du matin, mais "l'incendie avait déjà bien démarré", a-t-il expliqué. France Info précise que les victimes sont trois adultes et deux enfants. "L'origine de l'incendie est indéterminée mais a priori accidentelle", a déclaré le procureur adjoint de la République de Nîmes, cité par Europe 1.

Cinq personnes sont mortes, et treize autres ont été blessées à Nîme

In [229]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(article)
summ = nlp(summary)

sentences = []
cur_sentence = []
for sent in doc.sents:
    for token in sent:
        if not token.text.lower() in STOP_WORDS and not token.is_punct:
            cur_sentence.append(token.lemma_)
    sentences.append(cur_sentence)
    cur_sentence = []

for s in sentences:
    print(s)
    print()

['personne', 'mourir', 'être', 'blesser', 'Nîmes', 'Gard', 'incendie', 'déclencher', 'vendredi', 'premier', 'janvier', 'petit', 'matin']

['feu', 'ignore', 'origine', 'instant', 'prendre', 'dernier', 'étage', 'immeuble']

['arrivée', 'pompier', 'personne', 'décéder', 'appartement', 'appartement', 'voisin', 'intoxication', 'expliquer', 'Télé', 'directeur', 'cabinet', 'préfet', 'Gard']

['dénombre', 'blessé', 'grave']

['coma', 'être', 'transférer', 'Marseille', '-t', 'il', 'ajouté']

['secours', 'être', 'prévenir', '5', 'heure', 'matin', 'incendie', 'bien', 'démarrer', '-t', 'il', 'expliquer']

['France', 'Info', 'précise', 'victime', 'adulte', 'enfant']

['origine', 'incendie', 'indéterminer', 'priori', 'accidentel', 'déclarer', 'procureur', 'adjoint', 'république', 'nîme', 'citer', 'Europe', '1']



In [94]:
dictionary = corpora.Dictionary(sentences)
print(dictionary)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in sentences]
tfidf = models.TfidfModel(doc_term_matrix)
sentences_tfidf = tfidf[doc_term_matrix]
for sent in sentences_tfidf:
    print(sent)

Dictionary(64 unique tokens: ['Gard', 'Nîmes', 'blesser', 'déclencher', 'incendie']...)
[(0, 0.2131950688824599), (1, 0.31979260332368986), (2, 0.31979260332368986), (3, 0.31979260332368986), (4, 0.15083950856500844), (5, 0.31979260332368986), (6, 0.2131950688824599), (7, 0.31979260332368986), (8, 0.2131950688824599), (9, 0.31979260332368986), (10, 0.31979260332368986), (11, 0.31979260332368986), (12, 0.15083950856500844)]
[(13, 0.36650833306891567), (14, 0.36650833306891567), (15, 0.36650833306891567), (16, 0.36650833306891567), (17, 0.36650833306891567), (18, 0.24433888871261045), (19, 0.36650833306891567), (20, 0.36650833306891567)]
[(0, 0.17609018126512477), (8, 0.17609018126512477), (21, 0.26413527189768715), (22, 0.5282705437953743), (23, 0.26413527189768715), (24, 0.26413527189768715), (25, 0.26413527189768715), (26, 0.26413527189768715), (27, 0.17609018126512477), (28, 0.26413527189768715), (29, 0.26413527189768715), (30, 0.26413527189768715), (31, 0.26413527189768715)]
[(32, 0

In [121]:
coherence_values = []
model_list = []
for num_topics in range(2, 10):
    model = LsiModel(sentences_tfidf, num_topics=num_topics, id2word=dictionary)
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=sentences, dictionary=dictionary)
    coherence_values.append(coherencemodel.get_coherence())

print(coherence_values)
max_coherence = coherence_values.index(max(coherence_values))
num_topics = 2 + max_coherence
model = model_list[max_coherence]
print(num_topics)


[0.3191364025711909, 0.32969486342612486, 0.3577251118510679, 0.30675900745285184, 0.3274685716991899, 0.3347770831900468, 0.3311426885707415, 0.30609653827767835]
4


In [141]:
print(model.print_topics(num_topics=num_topics))
print()
corpus_lsi = model[doc_term_matrix]
for score, text in zip(corpus_lsi, doc.sents):
    print(score, text)


[(0, '-0.281*"-t" + -0.281*"il" + -0.260*"être" + -0.222*"matin" + -0.217*"ajouté" + -0.217*"transférer" + -0.217*"Marseille" + -0.217*"coma" + -0.205*"prévenir" + -0.205*"secours"'), (1, '-0.281*"origine" + -0.232*"dernier" + -0.232*"instant" + -0.232*"immeuble" + -0.232*"étage" + -0.232*"ignore" + -0.232*"feu" + -0.232*"prendre" + -0.189*"république" + -0.189*"accidentel"'), (2, '0.356*"appartement" + 0.210*"personne" + 0.210*"Gard" + -0.184*"transférer" + -0.184*"coma" + -0.184*"ajouté" + -0.184*"Marseille" + 0.178*"arrivée" + 0.178*"pompier" + 0.178*"préfet"'), (3, '-0.373*"précise" + -0.373*"Info" + -0.373*"adulte" + -0.373*"enfant" + -0.373*"France" + -0.373*"victime" + -0.236*"blessé" + -0.236*"dénombre" + -0.236*"grave" + 0.000*"origine"')]

[(0, -1.9391953183330903), (1, -0.6375553403983308), (2, 1.5240520935268955)] Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes, dans le Gard, dans un incendie qui s'est déclenché vendredi 1er janvier au petit matin.
[(0

In [183]:
# Sorts top sentence indices for each topic
top_scores = [[] for i in range(num_topics)]
for i, scores in enumerate(corpus_lsi): # i = sentence index in corpus
    for j, score in scores: # j = topic index
        top_scores[j].append((i, abs(score)))

for topic in top_scores:
    topic.sort(reverse=True, key=lambda x: x[1])

print()
for v in top_scores:
    print(v)



[(5, 2.6205547267103446), (0, 1.9391953183330903), (4, 1.6878013462424406), (2, 1.2384109509570864), (7, 0.5109342456404832), (1, 0.07596665115147677)]
[(7, 2.4565605363228467), (1, 1.9042651179474364), (2, 0.7474116198400329), (4, 0.6970255895590033), (0, 0.6375553403983308), (5, 0.34474953813756964)]
[(2, 2.8104549543671706), (0, 1.5240520935268955), (4, 1.1125870667135036), (1, 0.8223732250212842), (7, 0.8058302996737481), (5, 0.5926601566605476)]
[(6, 2.2361392455019122), (3, 0.7069940858048749)]


In [211]:
def get_top_sentences(summary_size=5):
    top_sentences = set()
    count = 0
    for i in range(summary_size):
        for j in range(num_topics):
            if i >= len(top_scores[j]):
                continue
            top_sentences.add(top_scores[j][i][0])
            if len(top_sentences) == summary_size:
                return sorted(top_sentences)
            
top_sentences = get_top_sentences()
print(top_sentences)

[0, 2, 5, 6, 7]


In [244]:
sents = list(doc.sents)
longest_summary = ''
for i in range(1, len(sents) + 1):
    top_sentences = get_top_sentences(i)
    summary = ""
    for sent_idx in top_sentences:
        #summary += sents[sent_idx].text + "\n"
        summary += ' '.join(word for word in sentences[sent_idx]) + "\n"
    if len(summary) > 280:
        break
    longest_summary = summary
if longest_summary:
    longest_summary = longest_summary[:-1]
    
print(longest_summary)

secours être prévenir 5 heure matin incendie bien démarrer -t il expliquer
origine incendie indéterminer priori accidentel déclarer procureur adjoint république nîme citer Europe 1


In [246]:
summ_sentences = []
summ_cur_sentence = []
for sent in summ.sents:
    for token in sent:
        if not token.text.lower() in STOP_WORDS and not token.is_punct:
            summ_cur_sentence.append(token.lemma_)
    summ_sentences.append(summ_cur_sentence)
    summ_cur_sentence = []


scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)

original_article = '\n'.join([sent.text for sent in doc.sents])
scores_original = scorer.score(original_article, longest_summary)

#reference_summary = '\n'.join([sent.text for sent in summ.sents])
reference_summary = '\n'.join([' '.join(sent) for sent in summ_sentences])
print(reference_summary)
scores_reference = scorer.score(reference_summary, longest_summary)

#print()
#print(longest_summary)
#print()
#print(original_article)
#print()
#print(reference_summary)
#print()
print(scores_original)
#print()
print(scores_reference)

personne mourir être blesser Nîmes Gard incendie déclencher vendredi premier janvier petit matin
{'rougeL': Score(precision=0.6774193548387096, recall=0.11475409836065574, fmeasure=0.19626168224299068)}
{'rougeL': Score(precision=0.0967741935483871, recall=0.2, fmeasure=0.13043478260869568)}
